In [13]:
import requests
import csv
import json
import time

# 🔧 CONFIGURATION
API_KEY = "cqt_rQ9PHYdJqwYdb63kCrGc9xPfwxck"
CHAIN_ID = 1  # Ethereum Mainnet
INPUT_FILE = r'C:\Users\HP\OneDrive\Desktop\wallets.csv'
OUTPUT_FILE = 'covalent_wallet_data1.json'

def load_wallets(file_path):
    with open(file_path, 'r') as file:
        reader = csv.reader(file)
        wallets = [row[0].strip().lower() for row in reader if row]
    return wallets

def fetch_wallet_transactions(wallet):
    url = f"https://api.covalenthq.com/v1/{CHAIN_ID}/address/{wallet}/transactions_v2/"
    params = {'key': API_KEY}
    try:
        response = requests.get(url, params=params)
        data = response.json()
        if 'data' in data and data['data']:
            print(f"✅ Data fetched for {wallet}")
            return {'wallet': wallet, 'data': data['data']}
        else:
            print(f"⚠️ No data found for {wallet}")
            return {'wallet': wallet, 'data': None}
    except requests.exceptions.RequestException as e:
        print(f"❌ Request failed for {wallet}: {str(e)}")
        return {'wallet': wallet, 'error': str(e)}

def main():
    wallets = load_wallets(INPUT_FILE)
    all_data = []

    for idx, wallet in enumerate(wallets):
        print(f"\nFetching {idx + 1}/{len(wallets)}: {wallet}")
        result = fetch_wallet_transactions(wallet)
        all_data.append(result)
        time.sleep(1.2)  # Avoid rate limit

    with open(OUTPUT_FILE, 'w') as outfile:
        json.dump(all_data, outfile, indent=2)

    print(f"\n✅ Done. Data saved to {OUTPUT_FILE}")

if __name__ == '__main__':
    main()



Fetching 1/104: ï»¿wallet_id
⚠️ No data found for ï»¿wallet_id

Fetching 2/104: 0x0039f22efb07a647557c7c5d17854cfd6d489ef3
✅ Data fetched for 0x0039f22efb07a647557c7c5d17854cfd6d489ef3

Fetching 3/104: 0x06b51c6882b27cb05e712185531c1f74996dd988
✅ Data fetched for 0x06b51c6882b27cb05e712185531c1f74996dd988

Fetching 4/104: 0x0795732aacc448030ef374374eaae57d2965c16c
✅ Data fetched for 0x0795732aacc448030ef374374eaae57d2965c16c

Fetching 5/104: 0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9
✅ Data fetched for 0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9

Fetching 6/104: 0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae
✅ Data fetched for 0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae

Fetching 7/104: 0x104ae61d8d487ad689969a17807ddc338b445416
✅ Data fetched for 0x104ae61d8d487ad689969a17807ddc338b445416

Fetching 8/104: 0x111c7208a7e2af345d36b6d4aace8740d61a3078
✅ Data fetched for 0x111c7208a7e2af345d36b6d4aace8740d61a3078

Fetching 9/104: 0x124853fecb522c57d9bd5c21231058696ca6d596
✅ Data fetched for 0x1

In [15]:
import json
import pandas as pd
import numpy as np
from datetime import datetime

# === Load JSON File ===
with open(r"C:\Users\HP\covalent_wallet_data1.json", "r") as f:
    wallets_data = json.load(f)

# === Store extracted features ===
features = []

# === Helper: Parse timestamps safely ===
def parse_timestamp(ts):
    try:
        return datetime.strptime(ts, "%Y-%m-%dT%H:%M:%SZ")
    except:
        return None

# === Loop through each wallet's data ===
for wallet_entry in wallets_data:
    wallet = wallet_entry.get("wallet")
    data = wallet_entry.get("data")
    if not data:
        continue

    items = data.get("items", [])
    if not items:
        continue

    # Initialize counters
    compound_txns = 0
    mint_events = 0
    usdc_minted = 0
    comp_rewards = 0
    gas_fees = []
    timestamps = []

    # === Process each transaction ===
    for tx in items:
        # Time
        block_time = parse_timestamp(tx.get("block_signed_at"))
        if block_time:
            timestamps.append(block_time)

        # Gas cost
        gas_fees.append(tx.get("gas_quote", 0) or 0)

        # Logs inside the transaction
        tx_logs = tx.get("log_events", [])

        for log in tx_logs:
            decoded = log.get("decoded")
            name = ""

            if isinstance(decoded, dict):
                name = decoded.get("name", "")

                # === Mint event (e.g., USDC to cUSDC) ===
                if name == "Mint":
                    params = decoded.get("params", [])
                    if isinstance(params, list):
                        for param in params:
                            if param.get("name") == "amount0":
                                try:
                                    usdc_minted += int(param.get("value", 0)) / 1e6  # USDC has 6 decimals
                                except:
                                    pass

                # === COMP distributed ===
                if name == "DistributedSupplierComp":
                    params = decoded.get("params", [])
                    if isinstance(params, list):
                        for param in params:
                            if param.get("name") == "compDelta":
                                try:
                                    comp_rewards += int(param.get("value", 0)) / 1e18  # COMP has 18 decimals
                                except:
                                    pass

            # === Check for Compound label ===
            sender_name = log.get("sender_name")
            if isinstance(sender_name, str) and "compound" in sender_name.lower():
                compound_txns += 1

    # === Skip wallets with no valid timestamps ===
    if not timestamps:
        continue

    # Calculate time span of activity
    min_time = min(timestamps)
    max_time = max(timestamps)
    time_span_days = (max_time - min_time).days or 1  # Avoid division by zero

    # Gas fee calculations
    avg_gas = np.mean(gas_fees) if gas_fees else 0
    total_gas = np.sum(gas_fees) if gas_fees else 0

    # Store all features
    features.append({
        "wallet_id": wallet,
        "compound_txns": compound_txns,
        "mint_events": mint_events,
        "usdc_minted": usdc_minted,
        "comp_rewards": comp_rewards,
        "avg_gas_fee": avg_gas,
        "total_gas_spent": total_gas,
        "time_span_days": time_span_days,
    })

# === Convert features to DataFrame ===
df = pd.DataFrame(features)

# === Normalize function (Min-Max) ===
def normalize(series):
    return (series - series.min()) / (series.max() - series.min() + 1e-9)

# === Normalize each feature ===
df["norm_mint"] = normalize(df["mint_events"])
df["norm_usdc"] = normalize(df["usdc_minted"])
df["norm_comp"] = normalize(df["comp_rewards"])
df["norm_avg_gas"] = 1 - normalize(df["avg_gas_fee"])  # lower gas = better
df["norm_time"] = normalize(df["time_span_days"])
df["norm_txns"] = normalize(df["compound_txns"])

# === Risk score (weighted sum of normalized values) ===
df["risk_score"] = 1000 * (
    0.25 * df["norm_mint"] +
    0.20 * df["norm_usdc"] +
    0.15 * df["norm_comp"] +
    0.15 * df["norm_avg_gas"] +
    0.15 * df["norm_time"] +
    0.10 * df["norm_txns"]
)

# Round & clip final risk score
df["risk_score"] = df["risk_score"].round().clip(0, 1000)

# === Export to CSV ===
df[["wallet_id", "risk_score"]].to_csv("wallet_risk_scores_using_normal_algorithm.csv", index=False)
print("✅ Risk scores saved to wallet_risk_scores.csv")


✅ Risk scores saved to wallet_risk_scores.csv
